In [2]:
# Addition (with BigInt and Int)
add(p::BigInt, a::BigInt, b::BigInt) = (a + b) % p
add(p, a, b) = add(BigInt(p), BigInt(a), BigInt(b))

# Subtraction
subtract(p::BigInt, a::BigInt, b::BigInt) = (a - b) % p
subtract(p, a, b) = subtract(BigInt(p), BigInt(a), BigInt(b))

# Multiplication
multiply(p::BigInt, a::BigInt, b::BigInt) = (a * b) % p
multiply(p, a, b) = multiply(BigInt(p), BigInt(a), BigInt(b))

# GCD
function gcd(a::BigInt, b::BigInt)
    r::BigInt = a % b
    if r == 0
        return b
    end
    return gcd(b, r)
end
gcd(a, b) = gcd(BigInt(a), BigInt(b))

# Extended Euclidian Algorithm
function extended_euclidian_alg(a::BigInt, b::BigInt)
    q = BigInt[]
    r::BigInt = a % b
    push!(q, div(a, b))
    while r != 0
        a = b
        b = r
        r = a % b
        push!(q, div(a, b))
    end
    return q
end
extended_euclidian_alg(a::Int64, b::Int64) = extended_euclidian_alg(BigInt(a), BigInt(b))

# Solving Linear Diophantine Equations
function solveLDE(a::BigInt, b::BigInt)
    P_n = BigInt(0)
    P_n_plus_one = BigInt(1)
    Q_n = BigInt(1)
    Q_n_plus_one = BigInt(0)
    q = extended_euclidian_alg(a, b)
    n = size(q)[1]
    parity = n % 2
    queue = 1
    while queue < n
        temp_P = P_n_plus_one
        next = q[queue]
        P_n_plus_one = P_n_plus_one * next + P_n
        P_n = temp_P
        temp_Q = Q_n_plus_one
        Q_n_plus_one = Q_n_plus_one * next + Q_n
        Q_n = temp_Q
        queue += 1
    end
    out = [P_n_plus_one Q_n_plus_one parity]
    return out
end

solveLDE(a, b) = solveLDE(BigInt(a), BigInt(b))

# Inverses
function inverse(p::BigInt, a::BigInt)
    solutions = solveLDE(p, a)
    return solutions[3] == 0 ? (p - solutions[1]) : solutions[1]
end

# Binary Representation (for fast powering algorithm)
function binary_representation(a::BigInt)
    out = Bool[]
    while a != 0
        next_bit = (a % 2) != 0
        pushfirst!(out, next_bit)
        a = div(a, 2)
    end
    return out
end

# Fast powering algorithm
function power(p::BigInt, a::BigInt, pow::BigInt)
    r = BigInt(1)
    for digit in binary_representation(pow)
        r = multiply(p, r, r)
        if digit
            r = multiply(p, r, a)
        end
    end
    return r
end

# Compositeness checker
function miller_rabin(p::BigInt)
    a::BigInt = rand(Int) % p
    if a == 1
        a = BigInt(rand(Int) % p)
    end
    k::BigInt = 0
    q::BigInt = p - 1
    while (q % 2 != 0)
        q = q % 2
        k += 1
    end
    a = BigInt(power(p, a, q))
    if a == 1
        return false
    end
    for i in 1:k
        if a == p - 1
            return false
        end
        a = multiply(p, a, a)
    end
    return true
end

# Primality Checker
function is_prime(p::BigInt)
    if p % 2 == 0
        return false
    end
    for i in 1:30
            if miller_rabin(p)
                return false
            end
        end
    return true
end

# Finds a candidate prime. 
function candidate(size::BigInt)
    primes = [2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97 101 103 107 109 113 127 131 137 139 149 151 157 163 167 173 181 191 193 197 199 211 223 227 229 233 239 241 251 257 263 269 271 277 281 283 293 307 311 313 317 331 337 347 349 353 359 367 373 379 383 389 397 401 409 419 421 431 433 439 443 449 457 461 463 467 479 487 491 499]
    multiple::BigInt = multiply(primes)
    rand_add::BigInt = multiply(primes)
    while (gcd(multiple, rand_add) != 1)
        rand_add = BigInt(rand(big"2":multiple))
    end
    k::BigInt = rand(big"1":big"2"^size)
    return BigInt((multiple * k) + rand_add)
end

# Multiplies elements of an array a together. 
function multiply(a)
    sum::BigInt = BigInt(1)
    for num in a
        sum = sum * BigInt(num)
    end
    return sum
end

# Finds a prime number of magnitude size
function find_prime(size::BigInt)
    while true
        can::BigInt = candidate(size::BigInt)
        if is_prime(can)
            return can
        end
    end
end

# Finds a prime number of magnitude size and a large ordered element
function find_prime_pair(size::BigInt)
    q = find_prime(size)
    while true
        factor = BigInt(rand(big"2":q))
        candidate = q * factor + big"1"
        if is_prime(candidate)
            return [q candidate factor]
        end
    end
end

# Generates a private and public key for ElGamal. 
function elgamal_key(p::BigInt, g::BigInt)
    print("p: \n")
    print(p)
    print("\ng: \n")
    print(g)
    a::BigInt = rand(big"2":p)
    public = power(p, g, a)
    print("\nA: \n")
    print(public)
    print("\na (Private!): \n")
    print(a)
end

# Generates a full set of ElGamal Keys. 
function elgamal_gen_keys()
    p = find_prime(BigInt(1000))
    g = rand(big"2":p)
    elgamal_key(p, g)
end

# Encrypts a message using the public key in ElGamal. 
function elgamal_encrypt(p::BigInt, g::BigInt, A::BigInt, m::BigInt)
    k::BigInt = rand(big"2":p)
    c_1::BigInt = power(p, g, k)
    print("C_1: \n")
    print(c_1)
    c_2::BigInt = multiply(p, m, power(p, A, k))
    print("\nC_2: \n")
    print(c_2)
end

# Decrypts the message in ElGamal
function elgamal_decrypt(p::BigInt, g::BigInt, a::BigInt, c_1::BigInt, c_2::BigInt)
    x::BigInt = power(p, c_1, a)
    m::BigInt = multiply(p, c_2, inverse(p, x))
    print("m: \n")
    print(m)
    return m
end

# Uses Baby-Step Big-Step to bash the discrete log problem for g^x = A mod p
function bash_discrete_log(p::BigInt, g::BigInt, A::BigInt)
    m::BigInt = ceil(p-1)
    tab = BigInt[]
    g_pow = 1
    for j = 1:m
        push!(tab, g_pow)
        g_pow = multiply(p, g_pow, g)
    end
    gamma = A
    for i = 0:(m-1)
        for idx = 1:m
            if gamma == tab[idx]
                return i * m + (idx - 1)
            end
        end
        gamma = multiply(p, gamma, inverse(power(p, a, m)))
    end
end

bash_discrete_log (generic function with 1 method)

In [18]:
save_key(big"1024")

q: 694b4c2e470916171bef7418f0c8b686ae94e91257f6913d3141a00ebd62b0bec6a02e92a9d9e1d5b4a3a04a257f4139614fdde8834f3bc981af578c9e6af80a9d989b8f57a1d2c4388c5cdffb10fa167513b195069acb65f607866f84317a64f7b786a3b1cbcc4ace1c9b1b5ae985738e0a795dc608bc6a98369d50ad85f34117e6d710a73bb2a3aa99ff161ee3de975c74cc885a91c2d97f17effc936371fdae6161d584b97b72943f5e02f328428438c242b1127ff9237258c1fc1aadb172e127eb68cdca6b854fa2c1d25de895fa82fda4ae1
p: 1848955509e84faf69c0162158bbf313c45b0daba21386383fe98e22575006748f78df34d1a2be238335f56520dc873a4c56609961d73784ad3e5d6d1575c98da365ebcd1ca21c06331d95b967ecda70f7a610757edd560474c74c62160dc38d2adc4f3a14d0abe6553890a51d2d9ad18a0f6e6ae2f11d3540412dc14c853bdfb378b30cf1da5f382f9a4efff809027d9a7229fce2360de1824ac5df917bd5db9e0646daa4fc45e9009fdedd52c41ec0d76894fd0a69e247ebeacb88e25933e718077687e498aadaf0dfdfe732f02d8e8085831dbf02155e9359f31b1d8824947dc008396fe09c743cd7cf43e0e4adfe5d59d54e4f2ac873c4a370f859e39159d4c475bfa6fde44152aa74e82f5b50ccf137e71dcd63861b8b3508d5

In [17]:
function save_key(size::BigInt)
    public = find_prime_pair(size)
    open("/tmp/elgamal_key.txt", "w") do f
        write(f, "[Generated ElGamal Keys: q, p, (p-1)/q, g, A (public), a (private)]\n\nq\n")
        write(f, string(public[1], base=16))
        println("q: ", string(public[1], base=16))
        write(f, "\n\np\n")
        write(f, string(public[2], base=16))
        println("p: ", string(public[2], base=16))
        write(f, "\n\n(p-1)/q\n")
        write(f, string(public[3], base=16))
        println("(p-1)/q: ", string(public[3], base=16))
        
    end
end

save_key (generic function with 1 method)

In [20]:
power(BigInt(228899585702368737238758122250774420677452887714971083850464281153676422584440316010389331986802043941571914922153943799695849076991799710783691534197818854127804725620844382614786069877404743022255917195749255701440191029652634676395768603521921590677864098512239760986435777597264465124327666981423755956973670460970748438906106708186207762722399033770221341630868659592487393608290007047678155630240206737324924080428660768256763018246027133508521768957394327716097766660670447830903494778161202978247788771114562613448385617533780025675437146561726710958407321246736264221216601737051462038563111980517329113770535947202195538079283368543496505909332811060987045998269907083980722815450205140466374466814711889628648087001866146855847071413409525471378334272588719981686778173421466463574512869230505411648485689463658376231504461820123173010941377199133637381142465185433125960571904556768364991757325963525330333928761929002194849045108895409032977627025568975617357715008241757450554624516024392364637835680441390179), BigInt(5), BigInt(9007837459444181738325279303510042427856869244190352528513032151568397173853701608390587136252599115990485906273920357213965541117563518430120424840954009364550699235625895623535452811504981265617930133319252548283044830550459681449942063091267199064285579122816821136222822952700923967362738173893301084957228825068889551089835496672918118907582942256357078580100808781834907465646224205743377706635994806232920145982180346301280940881648504499885919140044835376324385567514370041958540420353017513349205704054))

1799541124837191195083627390850757778207246123674390834083502545302619776505910396277082832633004481093569163360215775963366166612958913187135445586535241055125892573407677474157951091172682447193887446547957478575435105107612997868044366300096386720968913691808758937203599694150840862984865326743865936979518975797387476546777416940483776707372234922154367011761349825463863216998218658452207419775349115893617608836430332343735330480762852023424650458691734641277882190849269500336694747083997438448327323155638946507050002846149688438869490316188236171477793757355005882493036129142496364217543572155383871862604008543687303589267461632548415618581227625621630038445611222957959690679677259953173030294921291075893196011639006037914415602812903977042004052436202730087074348093568174917904197073613834886921610634760176857615610803033242597683618122774759124507077763013254865284518990899645892647568392812007580098083344856763514292398093995490648998694681308819785121908805427176373825701150052

In [21]:
elgamal_key(BigInt(760681717941856831493320494208068699140713615709244503770380357386684376451163275956323230773372045479498736854429937506339056773136102046458744157098410521433745533964035145783307274286672175365520411863658229193477406633825672621704053004446185290381871205754849060960807674317745831400623659116495750659568823365579547632949894086980592241843236320593668652662657694664132180825622168594880261903489873667483324930795035590547956395647058051136529829413096933294706420743213034951394917396407024909570661267632883851196108705655193853415548847489930230927776901820536478761605849576486751074177964880780177393483567699008868825281455829611008660051877250086776100150424967920297125606831988248093302916614768638681686825776219569530666588290436546553113957313179856477758951065545316623271950861851567111966863294279028191118552049005261884024390842194693299366339508455807123485655520254056476216550687434001755192433389531226439939278795282113143854079262420632212292062973281150597405303345825960440499710579146400639), BigInt(17995411248371911950836273908507577782072461236743908340835025453026197765059103962770828326330044810935691633602157759633661666129589131871354455865352410551258925734076774741579510911726824471938874465479574785754351051076129978680443663000963867209689136918087589372035996941508408629848653267438659369795189757973874765467774169404837767073722349221543670117613498254638632169982186584522074197753491158936176088364303323437353304807628520234246504586917346412778821908492695003366947470839974384483273231556389465070500028461496884388694903161882361714777937573550058824930361291424963642175435721553838718626040085436873035892674616325484156185812276256216300384456112229579596906796772599531730302949212910758931960116390060379144156028129039770420040524362027300870743480935681749179041970736138348869216106347601768576156108030332425976836181227747591245070777630132548652845189908996458926475683928120075800980833448567635142923980939954906489986946813088197851219088054271763738257011500527791527823571984673754))

p: 
760681717941856831493320494208068699140713615709244503770380357386684376451163275956323230773372045479498736854429937506339056773136102046458744157098410521433745533964035145783307274286672175365520411863658229193477406633825672621704053004446185290381871205754849060960807674317745831400623659116495750659568823365579547632949894086980592241843236320593668652662657694664132180825622168594880261903489873667483324930795035590547956395647058051136529829413096933294706420743213034951394917396407024909570661267632883851196108705655193853415548847489930230927776901820536478761605849576486751074177964880780177393483567699008868825281455829611008660051877250086776100150424967920297125606831988248093302916614768638681686825776219569530666588290436546553113957313179856477758951065545316623271950861851567111966863294279028191118552049005261884024390842194693299366339508455807123485655520254056476216550687434001755192433389531226439939278795282113143854079262420632212292062973281150597405303345

In [22]:
elgamal_decrypt(
    BigInt(760681717941856831493320494208068699140713615709244503770380357386684376451163275956323230773372045479498736854429937506339056773136102046458744157098410521433745533964035145783307274286672175365520411863658229193477406633825672621704053004446185290381871205754849060960807674317745831400623659116495750659568823365579547632949894086980592241843236320593668652662657694664132180825622168594880261903489873667483324930795035590547956395647058051136529829413096933294706420743213034951394917396407024909570661267632883851196108705655193853415548847489930230927776901820536478761605849576486751074177964880780177393483567699008868825281455829611008660051877250086776100150424967920297125606831988248093302916614768638681686825776219569530666588290436546553113957313179856477758951065545316623271950861851567111966863294279028191118552049005261884024390842194693299366339508455807123485655520254056476216550687434001755192433389531226439939278795282113143854079262420632212292062973281150597405303345825960440499710579146400639), 
    BigInt(17995411248371911950836273908507577782072461236743908340835025453026197765059103962770828326330044810935691633602157759633661666129589131871354455865352410551258925734076774741579510911726824471938874465479574785754351051076129978680443663000963867209689136918087589372035996941508408629848653267438659369795189757973874765467774169404837767073722349221543670117613498254638632169982186584522074197753491158936176088364303323437353304807628520234246504586917346412778821908492695003366947470839974384483273231556389465070500028461496884388694903161882361714777937573550058824930361291424963642175435721553838718626040085436873035892674616325484156185812276256216300384456112229579596906796772599531730302949212910758931960116390060379144156028129039770420040524362027300870743480935681749179041970736138348869216106347601768576156108030332425976836181227747591245070777630132548652845189908996458926475683928120075800980833448567635142923980939954906489986946813088197851219088054271763738257011500527791527823571984673754), 
    BigInt(641901694468680947681994703110755784611845976283216151198062607976068152034667114371404357756502682148395092646919400696297772679838484820809422948246508728575886058799854340177703999733516383532789200540507892023703434006991129842857281447835644071736343944550590274554686890624554404308346712438678526422576276457329260576773753268833762195427738933329865406582858775378946112761737074079367438380114061131170098188944663499240148149966248780196101874743863696660865812828838359013884136460489915900959791401106257710473461477285708727798270559491191357487513439551063915869823225215706016360523826186652144122402217764524697937111256662691023307398084741976661220125056575325659630866141740631543909034939964640635091180286040893542116677558002932656000292193506780493836299537771937910958464718650317330580724254131720311501205917516966305431152392385923673829901860203101439081899092040848050431399883560485303507753722307297972464791968366088277783175977050806630572542812947531973879739127679919464818824604616960557), 
    BigInt(567314014544748078119560436567963182605016115044648922962980146316506105386828357592393696795614620924049303273477832208028393984086364014966738916560277424962798753275782414887219697075287900827762756984968698244184452899560011202616489838392826309646538220054874796237132242259510693522252916732196817737334405319078812418579003830553863832652333885921711947736575492490980295674299388818970687474953319744677723500447093168646209968698018531524364968529412532734545060763399756999671921801477971554410434489062367520849766810926805919550383990714034799140664744478323289428870814708504281865255678443344631741670526511499739767830030612760522198535575294689748582384423743116967274956757409194319858927226079405820008260600522622977945962546912808018644088650445329291454800439698042044622916139590675290467320330020900557405684318077021861163297838408571242363025334675143398956498172617219912328205362407523583920924565371608507699892949712499908359964048630600257105149142733958443918796073541780170185045506920074236), 
    BigInt(663841178323487720522855340814005221443094069029926672945468032888002817605422259617196410679982995721484809811796122945472295528235604419536540371927646829322308319892459944091125361881437566527004499248653718623490465832892505673288852755390422140461507039513169112872076253453648679339082003574943233989333649589843596021426134115911107348032616106128042809025893863885948242474876885990933018155521132461808973688177987924677019789346177534989746608401979397641774648992284645702519278458578385368341209345845086010639671987392552943568687678699934563304296697306485845689783299568822202540603584493833073214541315640906816093612240956329780804938325926866010773267136918378754334518713688494897900477308898301754888337241833789967598409691164666452993964300845583068557134940726892114895803050348082521195871089844213814827363237944262752064063778260877805112454644224457416677059279166167948436890215095425091351868516351442863233827709913012311609555146259382756454518170818331131840812800321734537930933988483933121))

m: 
18004352591

18004352591